# Optimizing Strategy Parameters

In [ ]:
???

<table style="width:100%; height:90%">
      <tr>
    <th>Parametrize the Strategy</th>
    <th>Optimizing Limits' Parameters</th>
  </tr>
  <tr>
    <td><img src="src/07_Code_Regression Strategy Limits X.png" alt="Parametrize the Strategy" style="width:100%"></td>
    <td><img src="src/07_Table_Optimize BG Default Defaults.png" alt="Optimizing Limits' Parameters" style="width:100%"></td>
  </tr>
</table>

## Load the model

In [5]:
import pickle

with open('models/model_dt_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)
    
model_dt

DecisionTreeRegressor(max_depth=15)

## Load the data

In [6]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,
2016-12-08,56.325228,56.582507,55.902560,56.058762,21220800,1.549143,UP
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.321692,UP
2016-12-12,56.803028,57.244073,56.711145,57.124622,20198100,1.286112,UP
2016-12-13,57.427836,58.273172,57.188938,57.868881,35718900,-0.478622,DOWN
2016-12-14,57.887258,58.300739,57.455399,57.593227,30352700,-0.159789,DOWN
...,...,...,...,...,...,...,...
2023-03-09,255.820007,259.559998,251.580002,252.320007,26653400,-1.500467,DOWN
2023-03-10,251.080002,252.789993,247.600006,248.589996,28321800,2.099087,UP
2023-03-13,247.399994,257.910004,245.729996,253.919998,33339700,2.634307,UP


# Simple Investment Strategy

### Create Strategy class

In [7]:
from backtesting import Strategy, Backtest

In [8]:
class Regression(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > 1 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [9]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [10]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting with specific values

In [11]:
results = bt.run()

### Interpret backtesting results

In [12]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2023-03-15 00:00:00
Duration,2288 days 00:00:00
Exposure Time [%],84.390863
Equity Final [$],54328.698487
Equity Peak [$],59457.669965
Commissions [$],1674.630677
Return [%],443.286985


## Parametrize the Investment Strategy

### Create Strategy class

In [13]:
from backtesting import Strategy, Backtest

In [14]:
class Regression(Strategy):
    limit_buy = 1
    limit_sell = -5
    
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [15]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

## Optimize backtesting with multiple combinations

In [16]:
list_limits_buy = list(range(0, 11, 1))
list_limits_buy

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [17]:
list_limits_sell = list(range(0, -11, -1))
list_limits_sell

[0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10]

In [ ]:
%%time

results = bt.optimize(
    limit_buy = list_limits_buy, limit_sell = list_limits_sell,
    maximize='Return [%]', return_heatmap=True
)

C:\Users\zbppjx\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\_function_base_impl.py:552: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\zbppjx\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\_core\_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
C:\Users\zbppjx\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\backtesting\_stats.py:157: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov_matrix = np.cov(equity_log_returns, market_log_returns)
C:\Users\zbppjx\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\_function_base_impl.py:2894: RuntimeWarning: divide by zero encountered in 

In [ ]:
results[0]

In [ ]:
results[1]

### [ ] Interpret optimization results

In [ ]:
dff = results[1].reset_index()
dff

In [ ]:
dff = dff.pivot(index='limit_buy', columns='limit_sell', values='Return [%]')

### DataFrame heatmaps for better reporting

In [ ]:
dff.sort_index(axis=1, ascending=False)\
  .style.format(precision=0)\
    .background_gradient(vmin=dff.values.min(), vmax=dff.values.max())